In [1]:
import os
os.environ["CUDA_DEVICE_OREDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dataset
#from torchsummary import summary
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import random
print(torch.__version__, torchvision.__version__)
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)

1.8.0 0.2.2
TITAN RTX
11.1


In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
    
max_epochs = 30

In [3]:
# randomness 제어 
# https://hoya012.github.io/blog/reproducible_pytorch/
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)

In [5]:
vgg16 = torchvision.models.vgg16_bn(pretrained=False).to(device)
#summary(vgg16, input_size=(224,224,3))

In [6]:
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [7]:
# dataset load

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
dataset_path = "/media/2/Network/Imagenet_dup/"
# imagenet data load
train_dataset = dataset.ImageFolder(root=dataset_path+"train",
                                       transform=transform)
val_dataset = dataset.ImageFolder(root=dataset_path+"val",
                                       transform=transform)
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                        batch_size=16,
                                        shuffle=False,
                                        num_workers=0)
val_dataloader = torch.utils.data.DataLoader(val_dataset,
                                        batch_size=64,
                                        shuffle=False,
                                        num_workers=0)
print(len(train_dataloader),len(val_dataloader))

80073 782


In [8]:
len(train_dataloader),len(val_dataloader)

(80073, 782)

In [ ]:
# training phase
criterion = torch.nn.CrossEntropyLoss(size_average = None).to(device)
optimizer = torch.optim.SGD(vgg16.parameters(),lr=0.0005)
lr_sche = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

vgg16.to(device)
for epoch in range(max_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader):
        inputs,labels = data
        inputs,labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        y_pred = vgg16(inputs)
        # compute loss 
        loss = criterion(y_pred,labels)
        #print(epoch,i, loss.item())
        
        if not torch.isfinite(loss):
            print("WARNING: non-finite loss, ending training")
            exit(1)
        else :
            running_loss += loss.item()
        loss.backward()
        optimizer.step()
        if i % 30 == 29: 
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/30)) 
            running_loss = 0.0
            



        
        

[1,    30] loss: 0.936
[1,    60] loss: 0.025
[1,    90] loss: 1.410
[1,   120] loss: 0.101
[1,   150] loss: 0.019
[1,   180] loss: 1.449
[1,   210] loss: 0.045
[1,   240] loss: 0.017
[1,   270] loss: 1.443
[1,   300] loss: 0.027
[1,   330] loss: 1.340
[1,   360] loss: 0.227
[1,   390] loss: 0.020
[1,   420] loss: 1.389
[1,   450] loss: 0.033
[1,   480] loss: 0.013
[1,   510] loss: 1.425
[1,   540] loss: 0.027
[1,   570] loss: 0.579
[1,   600] loss: 0.899
[1,   630] loss: 0.017
[1,   660] loss: 1.332
[1,   690] loss: 0.048
[1,   720] loss: 0.013
[1,   750] loss: 1.343
[1,   780] loss: 0.027
[1,   810] loss: 0.011
[1,   840] loss: 1.353
[1,   870] loss: 0.022
[1,   900] loss: 1.429
[1,   930] loss: 0.119
[1,   960] loss: 0.016
[1,   990] loss: 1.471
[1,  1020] loss: 0.032
[1,  1050] loss: 0.012
[1,  1080] loss: 1.474
[1,  1110] loss: 0.025
[1,  1140] loss: 0.994
[1,  1170] loss: 0.663
[1,  1200] loss: 0.017
[1,  1230] loss: 1.375
[1,  1260] loss: 0.052
[1,  1290] loss: 0.013
[1,  1320] 